# 2-02、以DataFrame形式处理Socket流

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
lines = spark.readStream.format('socket').option('host', '192.168.89.130').option('port', '9999').load()

In [ ]:
type(lines)                

In [ ]:
lines.columns

In [ ]:
from pyspark.sql.functions import explode, split
words = lines.select(explode(split(lines.value, " ")).alias("word"))

In [ ]:
## 方法一：使用DataFrame API
wordCounts = words.groupBy("word").count() 

In [ ]:
##  方法二：使用Spark SQL
# words.createOrReplaceTempView("words_view")
# wordCounts = spark.sql('select word, count(word) from words_view group by word ')

In [ ]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()
# query.awaitTermination(180)

In [ ]:
query.stop()